# Burkley Dataset and Model

This notebook is from hugging face and first loads the Burkley dataset and then uses the models created by the authors to perform some analysis.


In [1]:
import datasets 
dataset = datasets.load_dataset('ucberkeley-dlab/measuring-hate-speech') 
df = dataset['train'].to_pandas()
df.describe()


/home/takosaga/miniconda3/envs/master_thesis/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,comment_id,annotator_id,platform,sentiment,respect,insult,humiliate,status,dehumanize,violence,...,hatespeech,hate_speech_score,infitms,outfitms,annotator_severity,std_err,annotator_infitms,annotator_outfitms,hypothesis,annotator_age
count,135556.000000,135556.000000,135556.000000,135556.000000,135556.000000,135556.00000,135556.000000,135556.000000,135556.000000,135556.000000,...,135556.000000,135556.000000,135556.000000,135556.000000,135556.000000,135556.000000,135556.000000,135556.000000,135556.000000,135451.000000
mean,23530.416138,5567.097812,1.281352,2.954307,2.828875,2.56331,2.278638,2.698575,1.846211,1.052045,...,0.744733,-0.567428,1.034322,1.001052,-0.018817,0.300588,1.007158,1.011841,0.014589,37.910772
std,12387.194125,3230.508937,1.023542,1.231552,1.309548,1.38983,1.370876,0.898500,1.402372,1.345706,...,0.932260,2.380003,0.496867,0.791943,0.487261,0.236380,0.269876,0.675863,0.613006,11.641276
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-8.340000,0.100000,0.070000,-1.820000,0.020000,0.390000,0.280000,-1.578693,18.000000
25%,18148.000000,2719.000000,0.000000,2.000000,2.000000,2.00000,1.000000,2.000000,1.000000,0.000000,...,0.000000,-2.330000,0.710000,0.560000,-0.380000,0.030000,0.810000,0.670000,-0.341008,29.000000
50%,20052.000000,5602.500000,1.000000,3.000000,3.000000,3.00000,3.000000,3.000000,2.000000,0.000000,...,0.000000,-0.340000,0.960000,0.830000,-0.020000,0.340000,0.970000,0.850000,0.110405,35.000000
75%,32038.250000,8363.000000,2.000000,4.000000,4.000000,4.00000,3.000000,3.000000,3.000000,2.000000,...,2.000000,1.410000,1.300000,1.220000,0.350000,0.420000,1.170000,1.130000,0.449555,45.000000
max,50070.000000,11142.000000,3.000000,4.000000,4.000000,4.00000,4.000000,4.000000,4.000000,4.000000,...,2.000000,6.300000,5.900000,9.000000,1.360000,1.900000,2.010000,9.000000,0.987511,81.000000


In [ ]:
## from user c-lonergan https://huggingface.co/ucberkeley-dlab/hate-measure-roberta-base/discussions/1#6642841ff057cd93085e4775
import datasets, transformers
from huggingface_hub import from_pretrained_keras
from tensorflow.keras.preprocessing.sequence import pad_sequences

dataset = datasets.load_dataset('ucberkeley-dlab/measuring-hate-speech', 'default', split='train')   
df = dataset.to_pandas()
# first 1000 rows
x = df['text'].unique()

model = from_pretrained_keras("ucberkeley-dlab/hate-measure-roberta-base")
model.compile()


tokenizer = transformers.RobertaTokenizer.from_pretrained("roberta-base")
tokens = tokenizer(x.tolist(), return_tensors='np', padding=True)

# padding as token shape is (None, 205), input shape is (None, 247)
padded_ids = pad_sequences(tokens['input_ids'], maxlen=247, padding='post', truncating='post')
padded_mask = pad_sequences(tokens['attention_mask'], maxlen=247, padding='post', truncating='post')
padded_inputs = [padded_ids, padded_mask]

y_pred = model.predict(padded_inputs)

2025-03-10 18:04:30.487707: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-10 18:04:30.517062: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-10 18:04:30.517098: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-10 18:04:30.517137: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-10 18:04:30.524008: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: A

   2/1237 [..............................] - ETA: 1:45:33

KeyboardInterrupt: 

2025-03-10 19:13:08,136 - INFO     - Executing shutdown due to inactivity...
2025-03-10 19:13:08,140 - INFO     - Executing shutdown...
2025-03-10 19:13:08,141 - INFO     - Not running with the Werkzeug Server, exiting by searching gc for BaseWSGIServer


In [3]:
df["hate_speech_score"]

0        -3.90
1        -6.52
2         0.36
3         0.26
4         1.54
          ... 
135551   -4.88
135552   -4.40
135553   -2.49
135554   -4.40
135555   -0.20
Name: hate_speech_score, Length: 135556, dtype: float64

In [4]:
# putting x and y_pred
import pandas as pd
preddf = pd.DataFrame({'text': x,'hate_speech_score': df["hate_speech_score"][0:100], 'pred_hate_score': y_pred.flatten()})


In [5]:
preddf

,text,hate_speech_score,pred_hate_score
0,Yes indeed. She sort of reminds me of the elde...,-3.90,-3.816762
1,The trans women reading this tweet right now i...,-6.52,-4.282561
2,Question: These 4 broads who criticize America...,0.36,0.192994
3,It is about time for all illegals to go back t...,0.26,0.001910
4,For starters bend over the one in pink and kic...,1.54,0.076700
...,...,...,...
95,Washington Post and there white genocide agenda.,-0.76,-0.647031
96,i almost puke seeing this transvestite,1.40,-0.141647
97,@nailainayat I thought in islam there are no s...,1.07,-0.632460
98,Maybe they are only good enough to get hood ni...,0.29,0.129705


In [6]:
df.head()

,comment_id,annotator_id,platform,sentiment,respect,insult,humiliate,status,dehumanize,violence,...,annotator_religion_hindu,annotator_religion_jewish,annotator_religion_mormon,annotator_religion_muslim,annotator_religion_nothing,annotator_religion_other,annotator_sexuality_bisexual,annotator_sexuality_gay,annotator_sexuality_straight,annotator_sexuality_other
0,47777,10873,3,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,False,False,False,False,False,False,False,False,True,False
1,39773,2790,2,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,False,False,False,False,False,False,False,False,True,False
2,47101,3379,3,4.0,4.0,4.0,4.0,4.0,4.0,0.0,...,False,False,False,False,True,False,False,False,True,False
3,43625,7365,3,2.0,3.0,2.0,1.0,2.0,0.0,0.0,...,False,False,False,False,False,False,False,False,True,False
4,12538,488,0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,False,False,False,False,False,False,False,False,True,False


In [2]:
import dtale
import pandas as pd


# Assigning a reference to a running D-Tale process.
d = dtale.show(df)


# Using Python's `webbrowser` package it will try and open your server's default browser to this process.
d.open_browser()



In [ ]:
import dtale

d = dtale.show(preddf)

d.open_browser()

Exception: data loaded must be one of the following types: pandas.DataFrame, pandas.Series, pandas.DatetimeIndex, pandas.MultiIndex, xarray.Dataset, numpy.array, numpy.ndarray, list, dict

In [13]:
d.kill()
dtale.instances()

/home/takosaga/miniconda3/envs/master_thesis/lib/python3.10/site-packages/dtale/app.py:445: FutureWarning:

`torch.distributed.reduce_op` is deprecated, please use `torch.distributed.ReduceOp` instead



currently no running instances...


In [15]:
# columns to keep
columns_to_keep = ["comment_id","text", "hate_speech_score"]
for column in df.columns.to_list():
    if column.startswith('target'):
        columns_to_keep.append(column)

In [16]:
columns_to_keep

['comment_id',
 'text',
 'hate_speech_score',
 'target_race_asian',
 'target_race_black',
 'target_race_latinx',
 'target_race_middle_eastern',
 'target_race_native_american',
 'target_race_pacific_islander',
 'target_race_white',
 'target_race_other',
 'target_race',
 'target_religion_atheist',
 'target_religion_buddhist',
 'target_religion_christian',
 'target_religion_hindu',
 'target_religion_jewish',
 'target_religion_mormon',
 'target_religion_muslim',
 'target_religion_other',
 'target_religion',
 'target_origin_immigrant',
 'target_origin_migrant_worker',
 'target_origin_specific_country',
 'target_origin_undocumented',
 'target_origin_other',
 'target_origin',
 'target_gender_men',
 'target_gender_non_binary',
 'target_gender_transgender_men',
 'target_gender_transgender_unspecified',
 'target_gender_transgender_women',
 'target_gender_women',
 'target_gender_other',
 'target_gender',
 'target_sexuality_bisexual',
 'target_sexuality_gay',
 'target_sexuality_lesbian',
 'target_